## Objectives of this section:
1. Basic exploration the data, get an understanding of the dataset
2. Clean the data, fill in missing info with appropriate values
3. More in depth exploration of the data, mainly analysing colinearity between variables and output to see if any columns can be dropped or combined. 
4. Check for any outliers in the data


In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import r2_score
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.append('/Users/ganeshsivam/Mods')
import corr

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

sns.set_style('darkgrid')
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [4]:
df = pd.read_csv('./data/train.csv')


In [5]:
nan_cols = df.columns[df.isna().any()].tolist()
no = []
for n in nan_cols:
    no.append(df.isnull().sum()[n])
nans = dict(zip(nan_cols,no))

In [6]:
nans

{'Lot Frontage': 330,
 'Alley': 1911,
 'Mas Vnr Type': 22,
 'Mas Vnr Area': 22,
 'Bsmt Qual': 55,
 'Bsmt Cond': 55,
 'Bsmt Exposure': 58,
 'BsmtFin Type 1': 55,
 'BsmtFin SF 1': 1,
 'BsmtFin Type 2': 56,
 'BsmtFin SF 2': 1,
 'Bsmt Unf SF': 1,
 'Total Bsmt SF': 1,
 'Bsmt Full Bath': 2,
 'Bsmt Half Bath': 2,
 'Fireplace Qu': 1000,
 'Garage Type': 113,
 'Garage Yr Blt': 114,
 'Garage Finish': 114,
 'Garage Cars': 1,
 'Garage Area': 1,
 'Garage Qual': 114,
 'Garage Cond': 114,
 'Pool QC': 2042,
 'Fence': 1651,
 'Misc Feature': 1986}

### 26 columns contin missing values, number of missing values go up to 1986 for the "Misc Feature" column

#### For these columns with NA values, we will assume that NA means the house doesn't have this feature. We can check this logic applies by comparing "Bsmt Qual" to "Total Bsmt SF". If "Bsmt Qual" is NA but "Total Bsmt SF" is not 0, we can conclude that something has gone wrong

In [7]:
df.loc[(df['Bsmt Qual'].isna())&(df["Total Bsmt SF"]>0)]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice


In [8]:
df.loc[(df['Mas Vnr Type'].isna())&(df["Mas Vnr Area"]>0)]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice


In [9]:
df.loc[(df['Garage Type'].isna())&(df["Garage Area"]>0)]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice


###### From the above filters, we can conclude that this method works fine. The following function implents this on the dataframe. Numeric columns are filled with 0 whilst object columns are filled with "No 'feature'"

One line of data has the Garage Blt Year as 2207, this is probably a typo for 2007 as that is the year the house was Remoded. We will make the change to this row

In [10]:
def fill_na(df):
    df["Lot Frontage"] = df["Lot Frontage"].fillna(df["Lot Frontage"].mean())
    df["Alley"] = df["Alley"].fillna("No alley access")
    df["Mas Vnr Type"] = df["Mas Vnr Type"].fillna("None")
    df["Mas Vnr Area"] = df["Mas Vnr Area"].fillna(0)
    #h = df.loc[df.loc[df['Garage Yr Blt']>2019]].index[0],'Garage Yr Blt']
    #df.loc[df['Garage Yr Blt']>2019]
    c = ['Bsmt Qual',
     'Bsmt Cond',
     'Bsmt Exposure',
     'BsmtFin Type 1',
     ]
    for i in df.index:
        if df.loc[i,"BsmtFin SF 1"] == 0:
            df.loc[i,c] = "No Basement"
        elif df.loc[i,"BsmtFin SF 1"] !=  df.loc[i,"BsmtFin SF 1"]:
            df.loc[i,c] = "No Basement"
            #df.loc[i,"BsmtFin SF 1"]
    c = ['BsmtFin Type 2']
    for i in df.index:
        if df.loc[i,"BsmtFin SF 2"] == 0:
            df.loc[i,c] = "No Basement"
        elif df.loc[i,"BsmtFin SF 2"] !=  df.loc[i,"BsmtFin SF 2"]:
            df.loc[i,c] = "No Basement"
        if df.loc[i,'BsmtFin Type 2'] != df.loc[i,'BsmtFin Type 2']:
            df.loc[i,'BsmtFin Type 2'] = df.loc[i,'BsmtFin Type 1']
    
    df['BsmtFin SF 1'] = df['BsmtFin SF 1'].fillna(0)
    df['BsmtFin SF 2'] = df['BsmtFin SF 2'].fillna(0)
    df['Bsmt Unf SF'] = df['Bsmt Unf SF'].fillna(0)
    df['Total Bsmt SF'] = df['Total Bsmt SF'].fillna(0)
    df['Bsmt Full Bath'] = df['Bsmt Full Bath'].fillna(0)
    df['Bsmt Half Bath'] = df['Bsmt Half Bath'].fillna(0)
    df['Fireplace Qu'] = df['Fireplace Qu'].fillna("No Fireplace")
    df['Garage Type'] = df['Garage Type'].fillna("No Garage")
    df['Garage Yr Blt'] = df['Garage Yr Blt'].fillna(-1)
    df['Garage Finish'] = df['Garage Finish'].fillna("No Garage")
    df['Garage Cars'] = df['Garage Cars'].fillna(0)
    df['Garage Area'] = df['Garage Area'].fillna(0)
    df['Garage Qual'] = df['Garage Qual'].fillna("No Garage")
    df['Garage Cond'] = df['Garage Cond'].fillna("No Garage")
    df['Pool QC'] = df['Pool QC'].fillna("No Pool")
    df['Fence'] = df['Fence'].fillna("No Fence")
    df['Misc Feature'] = df['Misc Feature'].fillna("No Misc")
    df.drop(['Id','PID'],axis=1,inplace=True)
    df['Garage Yr Blt'] = df['Garage Yr Blt'].astype(float)
    #try:
    #    h = df.loc[df['Garage Yr Blt']>2019].index[0]
    #    if h:
        
    #        df.loc[h,'Garage Yr Blt'] = df.loc[h,"Year Remod/Add"]
    #except:
    #    pass
    return df


According to the data dictionary, many columns are "Ordinal". According to Wikipedia, "The ordinal scale is distinguished from the nominal scale by having a ranking. It also differs from interval and ratio scales by not having category widths that represent equal increments of the underlying attribute."

The best way to handle these columns will be to conver them to numeric, i.e for a column with 5 different ranks, the ranks should be replaced by 0,1,2,3,4. This makes intuitive sense as now the model will be able to treat this data as ranked

In [11]:
mp1 =  {'Ex':4,'Gd':3,'TA':2,'Fa':1,'Po':0}
mpb = {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'No Basement':0}
mpbe = {'Gd':4,'Av':3,'Mn':2,'No':1,'No Basement':0}
mpbt = {'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,'No Basement':0}
funct = {'Typ':7,'Min1':6,'Min2':5,'Mod':4,'Maj1':3,
     'Maj2':2,'Sev':1,'Sal':0}
fp = {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'No Fireplace':0}
gf = {'Fin':3,'RFn':2,'Unf':1,'No Garage':0}
gq = {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'No Garage':0}
gc = {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'No Garage':0}
pool = {'Ex':4,'Gd':3,'TA':2,'Fa':1,'No Pool':0}
ls = {'Sev': 2, 'Mod': 1, 'Gtl': 0}
fence = {'GdPrv':4,'MnPrv':3,'GdWo':2,'MnWw':1,'No Fence':0}

ord_dict = {"Exter Qual":mp1,
           "Exter Cond":mp1,
           'Heating QC':mp1,
           'Kitchen Qual':mp1,
           'Bsmt Qual' : mpb,
           'Bsmt Cond': mpb,
           'Bsmt Exposure': mpbe,
            'BsmtFin Type 1':mpbt,
            'BsmtFin Type 2': mpbt,
            'Functional':funct,
            'Fireplace Qu': fp,
            'Garage Finish':gf,
            'Garage Qual':gq,
            'Garage Cond':gc,
            'Pool QC':pool,
            'Land Slope':ls,
            'Fence':fence
           }
def clean_ord(df,ord_dict): # Function to convert ordinal variables that are ranked for the model to deal with
    for key,value in ord_dict.items():
        df[key] = df[key].map(value)
    return df


In [12]:
df = fill_na(df)
df = clean_ord(df,ord_dict)


### Year columns should be coverted to "Years since". Only the "garage year built" column is problematic as sme entries do not have a garage. We will leave this out for now

In [13]:
year_cols = [c for c in df.columns if "Year" in c or "year" in c or 'Yr' in c]
year_cols = [c for c in year_cols if "rage" not in c]
    #year_cols.remove('Garage Yr Blt')
for c in year_cols:
    df[c] = 2019 - df[c]

In [14]:
corr = df.corr().sort_values('SalePrice',ascending=False)
corr = corr[['SalePrice']]
num_cols = list(corr.index)

In [15]:
def draw_scatters(df, variables):
    fig=plt.figure(figsize=(15,60))
    for i, var_name in enumerate(variables):
        ax=fig.add_subplot(round(len(variables)/2,0)+1,2,i+1)
        sns.regplot(x=var_name,y='SalePrice',data=df,fit_reg=False, scatter_kws={'alpha':0.2})
        ax.set_title(var_name +" vs. Sale Price")
    fig.tight_layout()
    #plt.savefig('../assets/png/scatters.png')
    plt.show()

In [16]:
#draw_scatters(df, num_cols)

#### The scatter plots suggest the data might have some outliers, particularly in the "Total Bsmt SF" & "GR Liv Area" columns. Let's check these out!

In [17]:
df.loc[df['Gr Liv Area']>5000]

,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
960,60,RL,313.0,63887,Pave,No alley access,IR3,Bnk,AllPub,Corner,0,Edwards,Feedr,Norm,1Fam,2Story,10,5,11,11,Hip,ClyTile,Stucco,Stucco,Stone,796.0,4,2,PConc,5,3,4,6,5644.0,0,0.0,466.0,6110.0,GasA,4,Y,SBrkr,4692,950,0,5642,2.0,0.0,2,1,3,1,4,12,7,3,4,Attchd,2008.0,3,2.0,1418.0,3,3,Y,214,292,0,0,0,480,3,0,No Misc,0,1,11,New,160000
1885,20,RL,128.0,39290,Pave,No alley access,IR1,Bnk,AllPub,Inside,0,Edwards,Norm,Norm,1Fam,1Story,10,5,11,10,Hip,CompShg,CemntBd,CmentBd,Stone,1224.0,4,2,PConc,5,3,4,6,4010.0,0,0.0,1085.0,5095.0,GasA,4,Y,SBrkr,5095,0,0,5095,1.0,1.0,2,1,2,1,4,15,7,2,4,Attchd,2008.0,3,3.0,1154.0,3,3,Y,546,484,0,0,0,0,0,0,Elev,17000,10,12,New,183850


In [18]:
df.loc[df['Total Bsmt SF']>5000]

,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
960,60,RL,313.0,63887,Pave,No alley access,IR3,Bnk,AllPub,Corner,0,Edwards,Feedr,Norm,1Fam,2Story,10,5,11,11,Hip,ClyTile,Stucco,Stucco,Stone,796.0,4,2,PConc,5,3,4,6,5644.0,0,0.0,466.0,6110.0,GasA,4,Y,SBrkr,4692,950,0,5642,2.0,0.0,2,1,3,1,4,12,7,3,4,Attchd,2008.0,3,2.0,1418.0,3,3,Y,214,292,0,0,0,480,3,0,No Misc,0,1,11,New,160000
1885,20,RL,128.0,39290,Pave,No alley access,IR1,Bnk,AllPub,Inside,0,Edwards,Norm,Norm,1Fam,1Story,10,5,11,10,Hip,CompShg,CemntBd,CmentBd,Stone,1224.0,4,2,PConc,5,3,4,6,4010.0,0,0.0,1085.0,5095.0,GasA,4,Y,SBrkr,5095,0,0,5095,1.0,1.0,2,1,2,1,4,15,7,2,4,Attchd,2008.0,3,3.0,1154.0,3,3,Y,546,484,0,0,0,0,0,0,Elev,17000,10,12,New,183850


The outliers are the same 2 rows. We will be better off removing them

In [19]:
def outliers_remove(df):
    df = df.loc[df['Gr Liv Area']<5000]
    return df
df = outliers_remove(df) 

### We can use bar charts to visualize the data for categorical features

In [20]:
def draw_bars(df, variables):
    fig=plt.figure(figsize=(20,100))
    for i, var_name in enumerate(variables):
        ax=fig.add_subplot(round(len(variables)/2,0)+1,2,i+1)
        sns.barplot(x=var_name,y='SalePrice',data=df,ci='sd')
        if var_name in ['neighborhood','exterior_1st','exterior_2nd']:
            plt.sca(ax)
            plt.xticks(rotation=70)
        ax.set_title(var_name)
    fig.tight_layout()
    
    plt.show()

In [21]:
cat_cols = [c for c in df.columns if df[c].dtype == "O"]
#draw_bars(df, cat_cols)

### Lastly for this section, we apply all functions to both train & test datasets and save as csv

In [22]:
def data_cleaning(df):
    df = fill_na(df)
    df = clean_ord(df,ord_dict)
    df = outliers_remove(df)
    #df = dummies(df)
    return df

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
test_id = test[['Id']]
test_id.to_csv("./data/test_id.csv",index=False)
train_cleaned = data_cleaning(train)
test_cleaned = data_cleaning(test)
train_cleaned.to_csv("./data/train_cleaned.csv",index=False)
test_cleaned.to_csv("./data/test_cleaned.csv",index=False)